# Deep learning model

In [265]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [266]:
# data loading from .npz generated from tcr_data_process.ipynb
# loading train data
npz = np.load('tcr_wop_train.npz')
train_inputs = npz['input'].astype(np.float)
train_targets = npz['targets'].astype(np.float)
# loading validation data
npz = np.load('tcr_wop_validation.npz')
validation_inputs, validation_targets = npz['input'].astype(np.float), npz['targets'].astype(np.float)

npz = np.load('tcr_wop_test.npz')
test_inputs, test_targets = npz['input'].astype(np.float), npz['targets'].astype(np.float)

C:\Users\parmarn\AppData\Local\Temp\ipykernel_14640\733935376.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['input'].astype(np.float)
C:\Users\parmarn\AppData\Local\Temp\ipykernel_14640\733935376.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_targets = npz['targets'].astype(np.float)
C:\Users\parmarn\Ap

# ANN Model

Outline, Optimizer, Loss Function, Early Stoping and Traing rate

In [267]:
# NN model prep
input_size =4
output_size = 2
hidden_layer_size = 8

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='linear')
])


In [268]:
# model prep, optimizer and loss function
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError(),metrics=[tf.keras.metrics.RootMeanSquaredError()])


In [269]:
# Training
BATCH_SIZE = 10
MAX_EPOCHS = 500
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

In [270]:
# fit the model
model.fit(
    train_inputs, train_targets, batch_size=BATCH_SIZE,epochs=MAX_EPOCHS, callbacks=[early_stopping], validation_data=(validation_inputs, validation_targets), verbose=2
          )

Epoch 1/500
5/5 - 1s - loss: 193.8762 - root_mean_squared_error: 13.9239 - val_loss: 98.2123 - val_root_mean_squared_error: 9.9102 - 837ms/epoch - 167ms/step
Epoch 2/500
5/5 - 0s - loss: 191.8661 - root_mean_squared_error: 13.8516 - val_loss: 96.8847 - val_root_mean_squared_error: 9.8430 - 50ms/epoch - 10ms/step
Epoch 3/500
5/5 - 0s - loss: 189.8800 - root_mean_squared_error: 13.7797 - val_loss: 95.5621 - val_root_mean_squared_error: 9.7756 - 60ms/epoch - 12ms/step
Epoch 4/500
5/5 - 0s - loss: 187.8550 - root_mean_squared_error: 13.7060 - val_loss: 94.2276 - val_root_mean_squared_error: 9.7071 - 57ms/epoch - 11ms/step
Epoch 5/500
5/5 - 0s - loss: 185.7717 - root_mean_squared_error: 13.6298 - val_loss: 92.8230 - val_root_mean_squared_error: 9.6345 - 55ms/epoch - 11ms/step
Epoch 6/500
5/5 - 0s - loss: 183.5893 - root_mean_squared_error: 13.5495 - val_loss: 91.3579 - val_root_mean_squared_error: 9.5581 - 55ms/epoch - 11ms/step
Epoch 7/500
5/5 - 0s - loss: 181.2305 - root_mean_squared_erro

# Model Test

In [271]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

1/1 [==============================] - 0s 34ms/step - loss: 2.1243 - root_mean_squared_error: 1.4575


In [279]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f} rmse'.format(test_loss, test_accuracy))


Test loss: 2.12. Test accuracy: 1.46 rmse


In [278]:
prediction = model.predict(test_inputs)
prediction = pd.DataFrame(prediction)
test_targets = pd.DataFrame(test_targets)

result = pd.concat([test_targets, prediction], axis = 1, ignore_index=True)
result.columns = ['Targets-dTi', 'Targets-dT','Prediction-dTi', 'Prediction-dT']
result

1/1 [==============================] - 0s 42ms/step


,Targets-dTi,Targets-dT,Prediction-dTi,Prediction-dT
0,12.0,0.333333,12.659404,1.614049
1,11.0,4.666667,9.115377,3.689142
2,14.0,6.666667,13.117580,5.843506
3,13.0,1.666667,12.067499,1.409785
4,14.0,11.333333,17.547916,12.803733
5,9.0,5.333333,9.269301,6.291492
6,16.0,0.666667,16.220861,2.897513
